In [ ]:
import xarray as xr
import datacube
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
import numpy as np
import dask
from datacube.drivers.netcdf import write_dataset_to_netcdf
from datacube.helpers import write_geotiff
from dask.distributed import Client
from src import DEADataHandling
from multiprocessing import Pool

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# client = Client(n_workers=6, threads_per_worker=1, memory_limit='4GB')
# client

In [ ]:
dc = datacube.Datacube(app='ndvi_range')

In [ ]:
lat, lon = -42.096, 147.403
latLon_adjust = 0.10
ncpus=4 #19

In [ ]:
def compute_range(time):
    query = {'lon': (lon - latLon_adjust, lon + latLon_adjust),
         'lat': (lat - latLon_adjust, lat + latLon_adjust),
        'time': time}
    
    ds = DEADataHandling.load_clearlandsat(dc=dc, query=query, sensors=['ls5','ls7','ls8'],  product='nbart',
                                       bands_of_interest=['nir', 'red'],mask_pixel_quality=True,
                                       mask_invalid_data=True)
   
    ndvi = ((ds.nir - ds.red)/(ds.nir + ds.red))
    min_ndvi = ndvi.min('time')
    max_ndvi = ndvi.max('time')
    range_ndvi = max_ndvi - min_ndvi

    range_ndvi = range_ndvi.rename('ndvi_range').to_dataset()
    range_ndvi.attrs = ds.attrs
    range_ndvi.attrs['units'] = 1
    range_ndvi.ndvi_range.attrs = ds.attrs
    range_ndvi.ndvi_range.attrs['units'] = 1

    write_geotiff("data/ndvi_range/ndvi_range_" + time[0][:-3] +'.tif', range_ndvi)

    

In [ ]:
x = []
for i in range(2000, 2019):
    x.append((str(i)+'-01', str(i)+'-12'))

In [ ]:
p = Pool(ncpus)
p.map(compute_range, x) # MULTIPROCESS

In [ ]:
xr.open_rasterio("data/ndvi_range/ndvi_range_2004.tif")